# 1 Start - pip install ucimlrepo / import / from [REQUIRED]

In [1]:
!pip install ucimlrepo # This is not included in Colab so you have to install it first

In [2]:
# Importing necessary libraries
import asyncio
import cProfile
import logging
import numpy as np
import math
import matplotlib.pyplot as plt
import os, sys
import pstats
import pandas as pd
import socket
import struct
import torch
import tornado
import traceback
#
from collections import Counter
from datetime import datetime
from typing import List
from math import sqrt
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from ucimlrepo import fetch_ucirepo
#
logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
#
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
#

# 2 Data Processing [REQUIRED]

### 2.1 Fetching the dataset using ucimlrepo package

In [3]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

### 2.2 Data Processing to X and y

In [4]:
# Putting dataset into X (features) and Y (target), X = what we are evaluating, y = the diagnosis
#
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets
#
# This will be used in FAM to define the number of features
Xfeaturenum = len(X.columns)
Xfeaturenum = int(Xfeaturenum)
#

In [5]:
# Show the columns
print(X.columns)

Index(['radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1',
       'compactness1', 'concavity1', 'concave_points1', 'symmetry1',
       'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 'area2',
       'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
       'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3',
       'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
       'symmetry3', 'fractal_dimension3'],
      dtype='object')


In [6]:
# Show X values
print(X)

     radius1  texture1  perimeter1   area1  smoothness1  compactness1  \
0      17.99     10.38      122.80  1001.0      0.11840       0.27760   
1      20.57     17.77      132.90  1326.0      0.08474       0.07864   
2      19.69     21.25      130.00  1203.0      0.10960       0.15990   
3      11.42     20.38       77.58   386.1      0.14250       0.28390   
4      20.29     14.34      135.10  1297.0      0.10030       0.13280   
..       ...       ...         ...     ...          ...           ...   
564    21.56     22.39      142.00  1479.0      0.11100       0.11590   
565    20.13     28.25      131.20  1261.0      0.09780       0.10340   
566    16.60     28.08      108.30   858.1      0.08455       0.10230   
567    20.60     29.33      140.10  1265.0      0.11780       0.27700   
568     7.76     24.54       47.92   181.0      0.05263       0.04362   

     concavity1  concave_points1  symmetry1  fractal_dimension1  ...  radius3  \
0       0.30010          0.14710     0.241

In [7]:
# Show y values
print(y)

    Diagnosis
0           M
1           M
2           M
3           M
4           M
..        ...
564         M
565         M
566         M
567         M
568         B

[569 rows x 1 columns]


### 2.3 Data Standardization (sklearn)

In [8]:
# Encode non numeric values to 1/0
label_encoder = LabelEncoder()
scalar = MinMaxScaler(feature_range=(0, 1))
X = scalar.fit_transform(X)
# Encode y to a 1d array using .ravel()
y_encoded = label_encoder.fit_transform(y.values.ravel())
# Now split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)


In [9]:
# Now show standardized X
print(X_train)

[[0.1452506  0.26445722 0.14249188 ... 0.22333333 0.26197516 0.14167651]
 [0.18074684 0.41494758 0.17275931 ... 0.25721649 0.27597083 0.14154532]
 [0.43348005 0.174163   0.41814664 ... 0.38797251 0.23910901 0.09891119]
 ...
 [0.11619102 0.29117349 0.11077327 ... 0.17360825 0.17524147 0.17263545]
 [0.12963226 0.28779168 0.11706171 ... 0.         0.06780997 0.06919848]
 [0.21434995 0.4808928  0.21235575 ... 0.33171821 0.10782574 0.21172767]]


In [10]:
# Now show standardized y
print (y_test)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1
 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0
 1 1 0]


# 3 ML with KNN SKLEARN [Optional]

### 3.1 Training and Predicting with the KNN model (sklearn)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_predk = classifier.predict(X_test)

In [12]:
# show the y.pred (predicted value) vs the y (actual value)
print(np.concatenate((y_predk.reshape(len(y_predk),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]]


### 3.2 Calculate and Print Metrics (KNN sklearn)

In [13]:
# Calculate KNN Metrics
accuracyk = accuracy_score(y_test, y_predk)
auc_rock = roc_auc_score(y_test, y_predk)
f1k = f1_score(y_test, y_predk)
precisionk = precision_score(y_test, y_predk)
sensitivityk = recall_score(y_test, y_predk)
tnk, fpk, fnk, tpk = confusion_matrix(y_test, y_predk).ravel()
specificityk = tnk / (tnk + fpk)

In [14]:
# Print KNN Metrics
print('### KNN Metrics ###')
print(f"Accuracy: {accuracyk}")
print(f"AUC-ROC: {auc_rock}")
print(f"F1 Score: {f1k}")
print(f"Precision: {precisionk}")
print(f"Sensitivity (Recall): {sensitivityk}")
print(f"Specificity: {specificityk}")
print(f"True Negatives: {tnk}")
print(f"False Positives: {fpk}")
print(f"False Negatives: {fnk}")
print(f"True Positives: {tpk}")

### KNN Metrics ###
Accuracy: 0.9649122807017544
AUC-ROC: 0.9574468085106382
F1 Score: 0.9555555555555556
Precision: 1.0
Sensitivity (Recall): 0.9148936170212766
Specificity: 1.0
True Negatives: 67
False Positives: 0
False Negatives: 4
True Positives: 43


# 4 ML with Fuzzy ARTMAP (FAM) [Optional]

### 4.1 FAM Tensor Data Prep

In [15]:
# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)  # Convert training input data to PyTorch tensor with float32 data type
y_train = torch.tensor(y_train, dtype=torch.float32)  # Convert training target data to PyTorch tensor with float32 data type
X_test = torch.tensor(X_test, dtype=torch.float32)    # Convert testing input data to PyTorch tensor with float32 data type
y_test = torch.tensor(y_test, dtype=torch.float32)    # Convert testing target data to PyTorch tensor with float32 data type

In [16]:
# Initialize FuzzyARTMAP
num_features = Xfeaturenum * 2 # Number of features in your data
num_categories = 2  # Number of unique categories in your data
# choice_param = 0.5  # Not really a hyperparameter, and should be much smaller see Carpenter et al., 1992
learning_rate = 0.1  # Hyperparameter, you may need to tune this
vigilance = 0.9  # Hyperparameter, you may need to tune this

### 4.2 FAM Class and Methods

### Fuzzy ARTMAP Class (Below)

**Description:**
The `FuzzyArtMap` class is an implementation of a Fuzzy ARTMAP neural network. This class is designed to perform pattern recognition and classification tasks. Fuzzy ARTMAP is particularly useful for tasks where the number of categories or classes can change dynamically. It can adapt to new categories without forgetting the previously learned ones.

**How It Works:**
- **Initialization Parameters:** The class takes various parameters during initialization, including the size of input and F2 layers, the number of categories, and vigilance criteria.
- **Resonance Search:** It uses the `_resonance_search_vector` method to find the most resonant F2 node for a given input pattern based on vigilance criteria.
- **Training:** The `train` method updates weights based on input patterns and their corresponding class vectors.
- **Prediction:** The `predict` method makes predictions for input patterns by finding the most resonant F2 node.
- **Complement Encoding:** It provides a static method, `complement_encode`, to perform complement encoding of input vectors.
- **Handling New Nodes:** The class handles the addition of new F2 nodes when needed.

**Why It's Needed:**
- Fuzzy ARTMAP is a versatile neural network that can adapt to new categories, making it suitable for tasks with changing or growing class sets.
- It provides a mechanism for incremental learning, allowing the network to remember and adapt to previously learned patterns.
- Fuzzy ARTMAP is used in various applications, including pattern recognition, classification, and adaptive learning.

**Components:**
- Input Layer (F1): Represents input patterns.
- F2 Layer: Contains category nodes for pattern recognition.
- Complement Coding: Utilizes complement encoding to enhance pattern representation.
- Vigilance Parameters: Control the matching criteria for resonance.
- Weight Matrices: Store connection weights between layers.
- Training and Prediction Methods: Implement the learning and prediction processes.



In [37]:
class FuzzyArtMap:
    def __init__(self,
                 f1_size: int = 10,
                 f2_size: int = 10,
                 number_of_labels: int = 2,
                 rho_a_bar = 0,
                 max_nodes = None,
                 committed_beta = 0.75, ):
        """
        Initialize Fuzzy ARTMAP instance and parameters
        Keyword arguments:
        f1_size - the number of elements in the compliment encoded vector (two-times the size of the input vector)
        f2_size - the initial number of coding category nodes - this will automatically grow up to max_nodes, unlimited if None
        number_of_labels - the compliment encoded number of labels for multi-label classification (e.g. for relevant, privilege this would be set to 4, for just relevant - 2)
        rho_a_bar - the baseline vigilence parameter to set the degree of match required to trigger resonance
        max_nodes - the maximum number of f2 nodes to grow to (Implementing the Max-Nodes mode of Carpenter, Grossberg, & Reynolds, 1995 - "A Fuzzy ARTMAP Nonparametric Probability Estimator for Nonstationary Pattern Recognition Problems")
        committed_beta - the learning rate for nodes that have already been commited (see Fast-Commit Slow-Recode Option in Carpenter et al., 1992)
        """
        if rho_a_bar < 0.0 or rho_a_bar > 1.0:
            raise ValueError(f"rho_a_bar must be between 0.0 and 1.0, received {rho_a_bar}")

        if committed_beta < 0.0 or committed_beta > 1.0:
            raise ValueError(f"committed_beta must be between 0.0 and 1.0, received {committed_beta}")

        self.alpha = 0.001  # "Choice" parameter > 0. Set small for the conservative limit (Fuzzy AM paper, Sect.3)
        self.dtype = torch.float
        self.beta = 1  # Learning rate. Set to 1 for fast learning
        self.beta_ab = 1 #ab learning rate - Enables Slow-lEarning mode from Carpenter, Grossberg, & Reynolds

        self.rho_ab = 0.95          # Map field vigilance, in [0,1]
        self.epsilon = 0.001        # Fab mismatch raises ARTa vigilance to this much above what is needed to reset ARTa
        self.device = torch.device("cpu")

        self.rho_a_bar = rho_a_bar  # Baseline vigilance for ARTa, in range [0,1]
        self.committed_beta = committed_beta
        self.max_nodes = max_nodes
        self.weight_a = torch.ones((f2_size, f1_size), device=self.device, dtype=self.dtype)
        self.input_vector_sum = f1_size / 2
        self.weight_ab = torch.ones((f2_size, number_of_labels), device=self.device, dtype=self.dtype)
        self.A_and_w = torch.empty(self.weight_a.shape, device=self.device, dtype=self.dtype)
        logger.info(f"f1_size: {f1_size}, f2_size:{f2_size}, committed beta = {self.committed_beta}")

        self.committed_nodes = set()
        self.updated_nodes = set()
        self.node_increase_step = 50 # number of F2 nodes to add when required
        self.number_of_increases = 0

    def _resonance_search_vector(self, input_vector: torch.tensor, already_reset_nodes: List[int], rho_a: float):
        """
        Search for a resonating F2 node based on input vector and vigilance criteria.

        Parameters:
        - input_vector: The input vector for which resonance is being searched.
        - already_reset_nodes: List of nodes that have already been reset.
        - rho_a: The vigilance parameter for ARTa.
        """
        resonant_a = False
        N, S, T = self.calculate_activation(input_vector)
        _, indices = torch.sort(T, stable=True, descending=True)
        all_membership_degrees = S / self.input_vector_sum
        T[already_reset_nodes] = torch.zeros((len(already_reset_nodes), ), dtype=self.dtype, device=self.device)
        while not resonant_a:
            for J in indices:
                if J.item() in already_reset_nodes:
                    continue

                if all_membership_degrees[J].item() >= rho_a or math.isclose(all_membership_degrees[J].item(), rho_a):
                    resonant_a = True
                    break
                else:
                    resonant_a = False
                    already_reset_nodes.append(indices[J].item())
                    T[indices[J].item()] = 0

            # Creating a new node if we've reset all of them
            if len(already_reset_nodes) >= N:
                if self.max_nodes is None or self.max_nodes >= (N + self.node_increase_step):
                    self.weight_a = torch.vstack((self.weight_a, torch.ones((self.node_increase_step,  self.weight_a.shape[1]), device=self.device, dtype=self.dtype)))
                    self.weight_ab = torch.vstack((self.weight_ab, torch.ones((self.node_increase_step, self.weight_ab.shape[1]), device=self.device, dtype=self.dtype)))
                    self.A_and_w = torch.vstack((self.A_and_w, torch.empty((self.node_increase_step,  self.weight_a.shape[1]), device=self.device, dtype=self.dtype)))
                    self.number_of_increases += 1
                else:
                    self.rho_ab = 0
                    self.beta_ab = 0.75
                    self.rho_a_bar = 0
                    rho_a = self.rho_a_bar
                    logger.info(f"Maximum number of nodes reached, {len(already_reset_nodes)} - adjusting rho_ab to {self.rho_ab} and beta_ab to {self.beta_ab}")
                    already_reset_nodes.clear()
                N, S, T = self.calculate_activation(input_vector)
                _, indices = torch.sort(T, stable=True, descending=True)
                all_membership_degrees = S / self.input_vector_sum
                T[already_reset_nodes] = torch.zeros((len(already_reset_nodes), ), dtype=self.dtype, device=self.device)

        return J.item(), all_membership_degrees[J].item()

    def calculate_activation(self, input_vector):
        """
        Calculate the activation levels for F2 nodes based on the input vector.

        Parameters:
        - input_vector: The input vector for which activation is calculated.
        """
        N = self.weight_a.shape[0]  # Count how many F2a nodes we have

        torch.minimum(input_vector.repeat(N,1), self.weight_a, out=self.A_and_w) # Fuzzy AND = min
        S = torch.sum(self.A_and_w, 1) # Row vector of signals to F2 nodes
        T = S / (self.alpha + torch.sum(self.weight_a, 1)) # Choice function vector for F2
        return N,S,T

    def train(self, input_vector: torch.tensor, class_vector: torch.tensor):
        """
        Train the FAM network with an input vector and class vector.

        Parameters:
        - input_vector: The input vector used for training.
        - class_vector: The class vector used for training.
        """
        rho_a = self.rho_a_bar # We start off with ARTa vigilance at baseline
        resonant_ab = False # Not resonating in the Fab match layer
        already_reset_nodes = [] # We haven't rest any ARTa nodes for this input pattern yet, maintain list between resonance searches of Fa

        class_vector = class_vector.to(self.device)
        input_vector = input_vector.to(self.device)
        class_vector_sum = torch.sum(class_vector, 1)
        while not resonant_ab:
            J, x = self._resonance_search_vector(input_vector, already_reset_nodes, rho_a)

            z = torch.minimum(class_vector, self.weight_ab[J, None])

            resonance = torch.sum(z, 1)/class_vector_sum
            if resonance > self.rho_ab or math.isclose(resonance, self.rho_ab):
                resonant_ab = True
            else:
                already_reset_nodes.append(J)
                rho_a = x + self.epsilon
                if rho_a > 1.0:
                    rho_a = 1.0 - self.epsilon

        self.updated_nodes.add(J)
        if J in self.committed_nodes:
            beta = self.committed_beta
        else:
            beta = self.beta

        self.weight_a[J, None] = (beta * torch.minimum(input_vector, self.weight_a[J, None])) + ((1-beta) * self.weight_a[J, None])
        self.weight_ab[J, None] = (self.beta_ab * z) + ((1-self.beta_ab) * self.weight_ab[J, None])
        self.committed_nodes.add(J)

    def fit(self, input_vectors, class_vectors):
        """
        Train the FAM network on a batch of input vectors and class vectors.

        Parameters:
        - input_vectors: A batch of input vectors.
        - class_vectors: A batch of class vectors.
        """
        for vector_index, input_vector in enumerate(input_vectors):
            self.train(input_vector, class_vectors[vector_index])
        self.updated_nodes.clear()


    @staticmethod
    def complement_encode(original_vector: torch.tensor) -> torch.tensor:
        """
        Complement encode the original input vector.

        Parameters:
        - original_vector: The original input vector to be complement-encoded.
        """
        complement = 1-original_vector
        complement_encoded_value = torch.hstack((original_vector,complement))
        return complement_encoded_value

    def predict(self, input_vector: torch.tensor):
        """
        Predict the class membership and membership degree for an input vector.

        Parameters:
        - input_vector: The input vector for which prediction is made.
        """
        rho_a = 0 # set ARTa vigilance to first match
        J, membership_degree = self._resonance_search_vector(input_vector, [], rho_a)

        # (Called x_ab in Fuzzy ARTMAP paper)
        return self.weight_ab[J, None], membership_degree # Fab activation vector & fuzzy membership value

    def save_model(self, descriptor: str):
        """
        Save the FAM model to a file with a timestamp and descriptor.

        Parameters:
        - descriptor: A descriptor for the saved model file.
        """
        model_timestamp = datetime.now().isoformat().replace("-", "_").replace(":", "_").replace(".", "_")
        cleaned_descriptor = descriptor.replace("-", "_").replace(":", "_").replace(".", "_")
        model_path = f"models/famgd_{model_timestamp}_{cleaned_descriptor}.pt"
        torch.save((self.weight_a, self.weight_ab, self.committed_nodes), model_path)
        return model_path

    def get_number_of_nodes(self):
        """
        Get the number of nodes in the F2 layer.
        """
        return self.weight_ab.shape[0]

    def get_number_of_increases(self) -> int:
        """
        Get the number of node increases during training.
        """
        return self.number_of_increases

    def get_increase_size(self) -> int:
        """
        Get the size of node increases during training.
        """
        return self.node_increase_step

    def get_committed_nodes(self) -> str:
        """
        Get the list of committed nodes.
        """
        return ",".join([str(n) for n in self.committed_nodes])

    def get_weight_a(self) -> torch.tensor:
        """
        Get the weight matrix of the F1 to F2 connections.
        """
        return self.weight_a

    def get_weight_ab(self) -> torch.tensor:
        """
        Get the weight matrix of the F2 to class connections.
        """
        return self.weight_ab

### Procedural Fuzzy ARTMAP Class (Below)

**Description:**
The `ProceduralFuzzyArtMap` class is an alternative implementation of a Fuzzy ARTMAP neural network using NumPy for computations. It serves the same purpose as the `FuzzyArtMap` class but uses a different approach.

**How It Works:**
- **Initialization Parameters:** This class accepts various parameters, such as layer sizes, vigilance criteria, and learning rates.
- **Resonance Search:** It uses the `_resonance_search` method to find the most resonant F2 node based on vigilance criteria.
- **Training:** The `train` method updates weights based on input patterns and class vectors.
- **Prediction:** The `predict` method makes predictions for input patterns.
- **Complement Encoding:** Provides a static method, `complement_encode`, for complement encoding.
- **Handling New Nodes:** The class appends new rows to weight matrices to add new nodes when required.

**Why It's Needed:**
- Procedural Fuzzy ARTMAP offers an alternative implementation for pattern recognition and classification tasks.
- It uses NumPy for computations, making it suitable for scenarios where PyTorch is not preferred or available.

**Components:**
- Input Layer (F1): Represents input patterns.
- F2 Layer: Contains category nodes for pattern recognition.
- Complement Coding: Utilizes complement encoding for enhanced pattern representation.
- Vigilance Parameters: Control the matching criteria for resonance.
- Weight Matrices: Store connection weights between layers.
- Training and Prediction Methods: Implement the learning and prediction processes.
- Category Growth Control: Allows or disallows the addition of new nodes based on the `allow_category_growth` parameter.

In [38]:
class ProceduralFuzzyArtMap:
    def __init__(self, f1_size: int = 10, f2_size: int = 10, number_of_categories: int = 2, rho_a_bar = 0):
        self.alpha = 0.001  # "Choice" parameter > 0. Set small for the conservative limit (Fuzzy AM paper, Sect.3)
        self.beta = 1  # Learning rate. Set to 1 for fast learning
        self.rho_a_bar = rho_a_bar  # Baseline vigilance for ARTa, in range [0,1]
        # use f1_size instead
        #self.M = size(a,1)  # Number of input components. Derived from data
                            # NB: Total input size = 2M (due to complement)
        # use f2_size instead
        # self.N = 20         # Number of available coding nodes
                            # We start with some resonably large number
                            # then, if we need to, can add more uncommitted
        # self.L = size(bmat,1)       # Number of output nodes. Derived from data ??? number output classes ????
        self.rho_ab = 0.95          # Map field vigilance, in [0,1]
        self.epsilon = 0.001        # Fab mismatch raises ARTa vigilance to this
                                    # much above what is needed to reset ARTa
        self.weight_a = np.ones((f2_size, f1_size)) # Initial weights in ARTa. All set to 1 Row-i, col-j entry = weight from input node i to F2 coding node j
        self.weight_ab = np.ones((f2_size, number_of_categories))  # Row-k, col-j entry = weight from ARTa F2  node j to Map Field node k
        self.committed_nodes = []

    def _resonance_search(self, input_vector, already_reset_nodes, rho_a, allow_category_growth = True):
        resonant_a = False
        while not resonant_a:
            # In search of a resonating ARTa node...
            # Find the winning, matching ARTa node

            N = self.weight_a.shape[0]
            # Count how many F2a nodes we have

            A_for_each_F2_node = input_vector * np.ones((N,1))
            # Matrix containing a copy of A for each F2 node. Useful for Matlab

            A_AND_w = np.minimum(A_for_each_F2_node, self.weight_a)
            # Fuzzy AND = min

            S = np.sum(A_AND_w, axis=1) # might be wrong operator
            # Row vector of signals to F2 nodes

            T = S / (self.alpha + np.sum(self.weight_a, axis=1))
            # Choice function vector for F2

            # Set all the reset nodes to zero
            T[already_reset_nodes] = np.zeros((len(already_reset_nodes), ), dtype=np.float32)

            # Finding the winning node, J

            J = np.argmax(T)
            # Matlab function max works such that J is the lowest index of max T elements, as
            # desired. J is the winning F2 category node

            # y = np.zeros((N, 1))
            # y[J]=1
            # Activities of F2. All zero, except J

            w_J = self.weight_a[J, :]  # ?????
            # Weight vector into winning F2 node, J

            x = np.minimum(input_vector, w_J)
            # Fuzzy version of 2/3 rule. x is F1 activity
            # NB: We could also use J-th element of S
            # since the top line of the match fraction
            # |I and w|/|I| is sum(x), which is
            # S = sum(A_AND_w) from above

            #####################################
            ######## Testing if the winning node resonates in ARTa

            if np.sum(x)/np.sum(input_vector) >= rho_a:
                # If a match, we're done
                resonant_a = True         # ARTa resonates
                # The while resonant_a == 0 command will stop looping
                # now, so we exit the while loop and go onto to Fab
            else:
                # If mismatch then we reset
                resonant_a = False     # So, still not resonating
                already_reset_nodes.append(J)
                # Record that node J has been reset already.

            #####################################
            # Creating a new node if we've reset all of them

            if len(already_reset_nodes) == N:
                if allow_category_growth:
                    # If all F2a nodes reset
                    self.weight_a = np.vstack((self.weight_a, np.ones((1, self.weight_a.shape[1]))))
                    self.weight_ab = np.vstack((self.weight_ab, np.ones((1, self.weight_ab.shape[1]))))
                else:
                    return -1, None
            # Give the new F2a node a w_ab entry
            # Now go back and this new node should win
        return J, x

    def train(self, input_vector: np.array, class_vector: np.array):
        rho_a = self.rho_a_bar
        # We start off with ARTa vigilance at baseline

        resonant_ab = False
        # Not resonating in the Fab match layer either

        already_reset_nodes = []  # We haven't rest any ARTa nodes for this input pattern yet

        while not resonant_ab:
            J, x = self._resonance_search(input_vector, already_reset_nodes, rho_a)

            # Desired output for input number i
            z = np.minimum(class_vector, self.weight_ab[J, :])   # Fab activation vector, z
            # (Called x_ab in Fuzzy ARTMAP paper)
            # Test for Fab resonance

            if np.sum(z)/np.sum(class_vector) >= self.rho_ab:     # We have an Fab match
                resonant_ab = True
            # This will cause us to leave the
            # while resonant_ab==0 loop and
            # go on to do learning.

            else: # We have an Fab mismatch
                resonant_ab = False
                # This makes us go through
                # the resonant_ab==0 loop again
                # resonant_a = False
                # This makes us go through
                # ARTa search again, this
                # search being inside the
                # resonant_ab==0 loop
                # Increase rho_a vigilance.
                # This will cause F2a node J to get reset when
                # we go back through the ARTa search loop again.
                # Also, *for this input*, the above-baseline
                # vigilance will cause a finer ARTa category to win

                rho_a = np.sum(x)/np.sum(input_vector) + self.epsilon

            # End of the while loop searching for ARTa resonance
            # If resonant_a = 0, we pick the next highest Tj
            # and see if *that* node resonates, i.e. goto "while"
            # If resonant_a = 1, we have found an ARTa resonance,
            # namely node J
            # So we go on to see if we get Fab match with node J

        #### End of the while resonant_ab==0 loop.
        #### Now we have a resonating ARTa output
        #### which gives a match at the Fab layer.
        #### So, we go on to have learning
        #### in the w_a and w_ab weights


        #### Let the winning, matching node J learn

        self.weight_a[J, :] = self.beta * x + (1-self.beta) * self.weight_a[J, :]
        # NB: x = min(A,w_J) = I and w
        #### Learning on F1a <--> f2a weights

        self.weight_ab[J, :] = self.beta * z + (1-self.beta) * self.weight_ab[J, :]
        # NB: z=min(b,w_ab(J))=b and w

    def predict(self, input_vector: np.array):
        rho_a = 0
        # We start off with ARTa vigilance at baseline
        resonant_a = False

        # We're not resonating in the ARTa module yet
        resonant_ab = False

        # Not resonating in the Fab match layer either
        already_reset_nodes = []  # We haven't rest any ARTa nodes for this input pattern yet

        while not resonant_ab:
            J, x = self._resonance_search(input_vector, already_reset_nodes, rho_a, False)

            # Desired output for input number i
            if J == -1:
                return np.zeros_like(self.weight_ab)

            z = self.weight_ab[J, None]   # Fab activation vector, z
            # prediction_transliteration = self.weight_ab[:,J]/sum(self.weight_ab[:,J])
            # (Called x_ab in Fuzzy ARTMAP paper)
            resonant_ab = True
            # prediction_transliteration = self.weight_ab[J,:]/np.sum(self.weight_ab[J,:])
            # print(prediction_transliteration)


            # End of the while loop searching for ARTa resonance
            # If resonant_a = 0, we pick the next highest Tj
            # and see if *that* node resonates, i.e. goto "while"
            # If resonant_a = 1, we have found an ARTa resonance,
            # namely node J
            # So we go on to see if we get Fab match with node J

        #### End of the while resonant_ab==0 loop.
        #### Now we have a resonating ARTa output
        #### which gives a match at the Fab layer.
        #### So, we go on to have learning
        #### in the w_a and w_ab weights


        #### Let the winning, matching node J learn

        # self.weight_a[:, J, np.newaxis] = self.beta * x + (1-self.beta) * self.weight_a[:, J, np.newaxis]
        # # NB: x = min(A,w_J) = I and w
        # #### Learning on F1a <--> f2a weights

        # self.weight_ab[J, :, np.newaxis] = self.beta * z + (1-self.beta) * self.weight_ab[J, :, np.newaxis]
        # NB: z=min(b,w_ab(J))=b and w
        return z

    @staticmethod
    def complement_encode(original_vector: np.array) -> np.array:
        # Calculate the complement of the original vector
        complement = 1 - original_vector

        # Stack the original vector and its complement horizontally
        complement_encoded_value = np.hstack((original_vector, complement))

        # Return the complement-encoded value
        return complement_encoded_value

## Fine-Tuning Fuzzy ARTMAP (FAM) Neural Networks

In this section, we'll explore the key features of Fuzzy ARTMAP (FAM) neural networks and how to fine-tune them for your classification tasks. FAM networks are versatile and can be adjusted to match your specific dataset and desired performance. Let's break down the tuning capabilities and their impact on FAM functionality:

### 1. Vigilance (rho_a_bar)

**Description:** Vigilance controls how selective FAM nodes are when accepting input patterns. Higher values make nodes more selective, while lower values make them more lenient.

**Impact:**
- **Increase:** Higher vigilance leads to finer-grained categories. Nodes accept only highly similar patterns.
- **Decrease:** Lower vigilance results in coarser-grained categories. Nodes accept a broader range of patterns.

**Standard Starting Value:** Typically set around 0.95.

**Tuning Based on Outputs:**
- If categories are too fine or many patterns are rejected, decrease vigilance.
- If categories are too broad or categories overlap, increase vigilance.

### 2. Learning Rate (beta)

**Description:** Learning rate (`beta`) determines how much weights adapt during learning. High values mean faster learning but can lead to instability. Low values result in slower but more stable learning.

**Impact:**
- **Increase:** Faster learning, but may be less stable.
- **Decrease:** Slower but more stable learning.

**Standard Starting Value:** Often set to 1 for fast learning.

**Tuning Based on Outputs:**
- If learning is too slow, increase beta.
- If the network is unstable or overshoots, decrease beta.

### 3. Map Field Vigilance (rho_ab)

**Description:** `rho_ab` controls F2 layer nodes' willingness to accept patterns from the ARTa layer.

**Impact:**
- **Increase:** Nodes in F2 become more selective.
- **Decrease:** Nodes in F2 become less selective.

**Standard Starting Value:** Typically set around 0.95.

**Tuning Based on Outputs:**
- If F2 nodes don't activate, decrease `rho_ab`.
- If F2 nodes activate too easily, increase `rho_ab`.

### 4. Node Growth (max_nodes)

**Description:** You can limit the maximum number of nodes in the F2 layer. When this limit is reached, you can choose to allow node growth or reset existing nodes.

**Impact:**
- **Node Growth:** Allows the network to adapt to new data by adding nodes.
- **Reset Nodes:** Resets existing nodes to accommodate new learning.

**Standard Starting Value:** Set to a reasonable maximum number of nodes (e.g., 20).

**Tuning Based on Outputs:**
- If categories become too rigid or new data isn't accommodated, allow node growth.
- If the network becomes too complex or unstable, reset nodes.

### 5. Other Parameters

Depending on the implementation, there may be additional parameters like epsilon, which adjusts vigilance based on mismatch.

Now that you understand these tuning capabilities, it's essential to monitor the network's performance and adjust these parameters based on the desired changes:
- Experiment with different values for each parameter.
- Observe how changes impact the network's ability to categorize patterns.
- Fine-tune until you achieve the desired categorization granularity and learning behavior.

Remember that the choice of parameter values should be data-dependent. Regularly assess the network's outputs and adapt your settings to optimize its performance for your specific classification task.


### 4.3 Initialize FAM

In [39]:
# Initialize the Fuzzy ARTMAP (FAM) model
fam = FuzzyArtMap(f1_size=num_features, number_of_labels=num_categories, committed_beta=learning_rate, rho_a_bar=vigilance)
pfam = ProceduralFuzzyArtMap(f1_size=num_features, number_of_categories=num_categories, rho_a_bar=vigilance)

### 4.4 FAM Train and Test

In [40]:
# Train FuzzyARTMAP
for i in range(len(X_train)):
    input_vector = FuzzyArtMap.complement_encode(X_train[i])
    target_vector = FuzzyArtMap.complement_encode(y_train[i, None, None])
    # Execute FAM Training
    fam.train(input_vector, target_vector)
    pfam.train(input_vector.numpy(), target_vector.numpy())

In [41]:
# Test FuzzyARTMAP
y_predf = []
correct_predictions = 0
for i in range(len(X_test)):
    #input_vector = X_test[i]
    input_vector = FuzzyArtMap.complement_encode(X_test[i])
    # switch these to use the streamlined vs. procedural version
    category = pfam.predict(input_vector.numpy())
    # category, _ = fam.predict(input_vector)
    y_predf.append(category[0,0])
    if category[0,0] == y_test[i]:
        correct_predictions += 1
# Now, 'predictions' contains the predicted categories for each input in X_test

In [ ]:
print(y_predf)

### 4.5 FAM Calculate and Print Metrics

In [42]:
# Calculate FAM Metrics
accuracyf = accuracy_score(y_test, y_predf)
auc_rocf = roc_auc_score(y_test, y_predf)
f1f = f1_score(y_test, y_predf)
precisionf = precision_score(y_test, y_predf)
sensitivityf = recall_score(y_test, y_predf)
tnf, fpf, fnf, tpf = confusion_matrix(y_test, y_predf).ravel()
specificityf = tnf / (tnf + fpf)

In [ ]:
# Print FAM Metrics
print('### FAM Metrics ###')
print(f"Accuracy: {accuracyf}")
print(f"AUC-ROC: {auc_rocf}")
print(f"F1 Score: {f1f}")
print(f"Precision: {precisionf}")
print(f"Sensitivity (Recall): {sensitivityf}")
print(f"Specificity: {specificityf}")
print(f"True Negatives: {tnf}")
print(f"False Positives: {fpf}")
print(f"False Negatives: {fnf}")
print(f"True Positives: {tpf}")

# 5 Model Comparison Table [Optional]

In [ ]:
# Print Metrics Comparison Table
# Define Table width and Title
tablewidth = 85
comparison_title = "Model Comparison: KNN vs FAM"
padding = "#" * ((tablewidth - len(comparison_title)) // 2)
centered_title = f"{padding} {comparison_title} {padding}"
# Print Table amd Column Titles
print(centered_title) # Table Title
print("-" * tablewidth)  # Line
print(f"{'Metric':<25} {'KNN':<20} {'FAM':<20} {'Diff'}")  # Define Column Titles
print("-" * tablewidth)  # Line
# Print Table Data
print(f"{'Accuracy:':<25} {accuracyk:<20.3f} {accuracyf:<20.3f} {accuracyk - accuracyf:.3f}")  # Calculate and display the difference
print(f"{'AUC-ROC:':<25} {auc_rock:<20.3f} {auc_rocf:<20.3f} {auc_rock - auc_rocf:.3f}")  # Calculate and display the difference
print(f"{'F1 Score:':<25} {f1k:<20.3f} {f1f:<20.3f} {f1k - f1f:.3f}")  # Calculate and display the difference
print(f"{'Precision:':<25} {precisionk:<20.3f} {precisionf:<20.3f} {precisionk - precisionf:.3f}")  # Calculate and display the difference
print(f"{'Sensitivity (Recall):':<25} {sensitivityk:<20.3f} {sensitivityf:<20.3f} {sensitivityk - sensitivityf:.3f}")  # Calculate and display the difference
print(f"{'Specificity:':<25} {specificityk:<20.3f} {specificityf:<20.3f} {specificityk - specificityf:.3f}")  # Calculate and display the difference
print(f"{'True Negatives:':<25} {tnk:<20.3f} {tnf:<20.3f} {tnk - tnf:.3f}")  # Calculate and display the difference
print(f"{'False Positives:':<25} {fpk:<20.3f} {fpf:<20.3f} {fpk - fpf:.3f}")  # Calculate and display the difference
print(f"{'False Negatives:':<25} {fnk:<20.3f} {fnf:<20.3f} {fnk - fnf:.3f}")  # Calculate and display the difference
print(f"{'True Positives:':<25} {tpk:<20.3f} {tpf:<20.3f} {tpk - tpf:.3f}")  # Calculate and display the difference